# Análisis Numérico I - FIUBA (Cátedra Tarela)
# Ecuaciones No Linealesb

## 1er cuatrimestre 2020

In [1]:
import numpy as np
import sympy as sp
import math
import matplotlib.pyplot as plt
from sympy.calculus.util import continuous_domain
from scipy.linalg import solve
from scipy.integrate import quad

# 0. Funciones auxiliares

### 0.1 Funciones auxiliares - Matrices

In [2]:
def print_matriz(matriz, spacing=15, rounding=-1):
    
    space_str = "{: >" + str(spacing)+ "}" + " "*3 + "\t"
    line = space_str * len(matriz)
    
    for fila in matriz:
        
        fila_modificada = fila
        if (rounding!=-1):
            fila_modificada = [round(x, rounding) for x in fila_modificada]  
            
        fila_string = [str(x) for x in fila_modificada]        
        print('|'+ line.format(*fila_string) + "|")


In [3]:
def print_vector(vector, spacing=4, rounding=-1):
    
    space_str = "{: >" + str(spacing)+ "}" + " "*3
    line = space_str * len(vector)
    
    vector_modificado = vector
    if (rounding!=-1):
            vector_modificado = [round(x, rounding) for x in vector_modificado]  
    
    vector_string = [str(x) for x in vector_modificado]
    
    print('|'+ line.format(*vector_string) + "|")
            

In [4]:
def producto_vectorial(v1, v2):
    
    n = len(v1)
    
    if(len(v2) != n): return None
    
    res = 0
    
    for i in range(n):
        res += v1[i]*v2[i]
    
    return res

In [5]:
def resolver_sistema_ecuaciones(A,b):
    A_bis = np.array(A, dtype = 'float')
    b_bis = np.array(b, dtype = 'float')
    return list(solve(A_bis,b_bis))

# 1. APROXIMACIÓN DE FUNCIONES

# 1.1  Aproximáción de funciones (discreta)

### 1.1.1 Cálculo del polinomio

In [6]:
def find_phis(phi, x):
    
    phis = []

    for f in phi:
        vect = []
        for n in x:
            vect.append(f(n))
        phis.append(vect)
    
    return phis

In [7]:
def matriz_phixphi(phis):

    n = len(phis)

    matriz  = []

    for fil in range(n):
        fila = []
        for col in range(n):
            fila.append(producto_vectorial(phis[fil], phis[col]))
        matriz.append(fila)
        
    return matriz

In [8]:
def vector_f_t(phis, y):
    f_t = []

    for p in phis:
        f_t.append(producto_vectorial(y,p))
        
    return f_t

In [9]:
def get_función_aproximacion(phi,c):
    
    x_symbol = sp.Symbol('x')
    phi_symbol = [p(x_symbol) for p in phi]
    res = 0
    for i in range(len(c)):
            res += c[i] * phi_symbol[i]
            
    return sp.lambdify(x_symbol, res, "math")

In [10]:
def aproximacion_cm(x,y,phi):
    
    phis = find_phis(phi, x)
    A = matriz_phixphi(phis)
    b = vector_f_t(phis, y)
    c = resolver_sistema_ecuaciones(A, b)
    P = get_función_aproximacion(phi,c)
    
    return phis,A,b,c,P

In [11]:
def info_aproximacion_cm(A, b, c, phi):
        
    print("\nAjk:")
    print_matriz(A, rounding=5)
    print()
    
    print("\nbk:")
    print_vector(b, rounding=5)
    print()
    
    print("\nC_k:")
    print_vector(c, rounding=5)
    print()
    
    print("\nAPROXIMACION POR CUADRADOS MÍNIMOS:\n")
    x_symbol = sp.Symbol('x')
    phi_symbol = [p(x_symbol) for p in phi]
    res = 0
    for i in range(len(c)):
            res += c[i] * phi_symbol[i]
    print(res)
    print("\n")

### 1.1.2 Cálculo del error

In [12]:
ERROR_ROUNDING = 3

In [13]:
def error_absoluto_aprox_discreta(f,y,x):
    e = [y[i]-f(x[i]) for i in range(len(x))]
    return e

In [14]:
def error_absoluto_cuadratico_aprox_discreta(f,y,x):
    e_absoluto = error_absoluto_aprox_discreta(f,y,x)
    return round(sum([e**2 for e in e_absoluto]),ERROR_ROUNDING)

In [15]:
def norma_error_absoluto_aprox_discreta(f,y,x):
    e_absoluto = error_absoluto_aprox_discreta(f,y,x)
    return round(sum([e**2 for e in e_absoluto])**(1/2),ERROR_ROUNDING)

In [16]:
def error_relativo_aprox_discreta(f,y,x):
    norma_error_absoluto = error_absoluto_cuadratico_aprox_discreta(f,y,x)**(1/2)
    norma_y =  sum([y_i**2 for y_i in y])**(1/2)
    return round(norma_error_absoluto/norma_y,ERROR_ROUNDING)

In [17]:
def info_errores_aprox_discreta(P,y,x):
    print("ERRORES\n")
    print("- error absoluto:")
    print_vector(error_absoluto_aprox_discreta(P,y,x), rounding=3)
    print("- error absoluto cuadratico:", error_absoluto_cuadratico_aprox_discreta(P,y,x))
    print("- ||e||:", norma_error_absoluto_aprox_discreta(P,y,x))
    print("- error relativo %:", error_relativo_aprox_discreta(P,y,x)*100,"%")

### 1.1.3 Ejemplos

#### Ejemplo 1

In [18]:
# Ejemplo 1 teórica

x = [-3/2, -1, -1/2, 0, 1/2, 1, 3/2]
y = [2.4, 1.4, 1.1, 0.9, 1.2, 1.5, 2.3]

phi =[lambda x: 1, lambda x: sp.exp(x), lambda x: sp.exp(-x)]

phis,A,b,c,P = aproximacion_cm(x,y,phi)

info_aproximacion_cm(A, b, c, phi)
info_errores_aprox_discreta(P,y,x)


Ajk:
|              7   	       11.04623   	       11.04623   	|
|       11.04623   	       31.74588   	7.00000000000000   	|
|       11.04623   	7.00000000000000   	       31.74588   	|


bk:
|10.8   18.98150   19.06810   |


C_k:
|-0.06803   0.50866   0.51216   |


APROXIMACION POR CUADRADOS MÍNIMOS:

0.508660004618175*exp(x) - 0.0680298360835958 + 0.512159439794367*exp(-x)


ERRORES

- error absoluto:
|0.059   -0.111   0.015   -0.053   0.119   -0.003   -0.026   |
- error absoluto cuadratico: 0.034
- ||e||: 0.184
- error relativo %: 4.3 %


#### Ejemplo 2

In [19]:
# Ejemplo entrega COVID-19 Chile

x = [1,10,20,30,40,50,56]
y = [1,10,114,373,286,464,552]

phi = [lambda x: 1, lambda x: x, lambda x: x**2, lambda x: x**3]

phis,A,b,c,P = aproximacion_cm(x,y,phi)
info_aproximacion_cm(A, b, c, phi)
info_errores_aprox_discreta(P,y,x)


Ajk:
|              7   	            207   	           8637   	         400617   	|
|            207   	           8637   	         400617   	       19624497   	|
|           8637   	         400617   	       19624497   	      993231777   	|
|         400617   	       19624497   	      993231777   	    51355979457   	|


bk:
|1800   79123   3730973   184237033   |


C_k:
|-27.13891   6.4914   0.13272   -0.00125   |


APROXIMACION POR CUADRADOS MÍNIMOS:

-0.0012482600842409*x**3 + 0.132722812153222*x**2 + 6.49139936415258*x - 27.138912396947


ERRORES

- error absoluto:
|21.516   -39.799   -31.792   119.649   -78.985   -9.206   18.616   |
- error absoluto cuadratico: 24043.556
- ||e||: 155.06
- error relativo %: 17.9 %


#### Ejemplo 3

In [20]:
# Ejemplo aproximación funcion potencial en escala logarítmica

x = [1,10,20,30,40,50,60]
y = [1,1.75,2.5,5,10,20,30]

log_x = [sp.ln(x_i) for x_i in x]
log_y = [sp.ln(y_i) for y_i in y]

phi = [lambda x: a*x**P]
log_phi = [lambda x: 1, lambda x: x]

phis,A,b,c,P = aproximacion_cm(log_x,log_y,log_phi)
info_aproximacion_cm(A, b, c, log_phi)
info_errores_aprox_discreta(P,log_x,log_y)


Ajk:
|              7   	       20.39476   	|
|       20.39476   	       71.51987   	|


bk:
|11.78486   43.64655   |


C_k:
|-0.5586   0.76956   |


APROXIMACION POR CUADRADOS MÍNIMOS:

0.769563247492521*x - 0.558599988440562


ERRORES

- error absoluto:
|0.559   2.431   2.849   2.721   2.475   2.165   2.036   |
- error absoluto cuadratico: 36.702
- ||e||: 6.058
- error relativo %: 71.6 %


# 1.2  Aproximáción de funciones (continua)

### 1.2.1 Cálculo del polinomio

In [21]:
def matriz_Hilbert(a, b, grado):
    
    matriz = []
    n = grado + 1
    
    for j in range(n):
        fila = []
        for k in range(n):
            p = j+k+1
            fila.append((b**p-a**p)/p)
        matriz.append(fila)
        
    return matriz
    

In [22]:
def b_Hilbert(f, grado, a, b):
    
    res = []
    
    for i in range(grado+1):
        integrand = lambda x: f(x) * (x**i)
        r,err = quad(integrand, a, b)
        res.append(r)
        
    return res

In [23]:
def polinomio(grado):
    return lambda x: x**grado

def funcione_polinomicas(grado):
    res = [polinomio(g) for g in range(grado+1)]
    return res

In [24]:
def aproximacion_cm_continua(f, grado, intervalo):
    
    a = intervalo[0]
    b = intervalo[1]
    
    phi = funcione_polinomicas(grado)
    A = matriz_Hilbert(a, b, grado)
    b = b_Hilbert(f, grado, a, b)
    c = resolver_sistema_ecuaciones(A, b)
    f = get_función_aproximacion(phi,c)
        
    return phi,A,b,c,P

### 1.2.2 Cálculo del error

In [25]:
def error_globlal_aprox_continuo(f, P, intervalo):
    a = intervalo[0]
    b = intervalo[1]
    
    integrand = lambda x: (f(x) - P(x))**2
    r,err = quad(integrand, a, b)
    return r

In [26]:
def error_relativo_aprox_continuo(f, P, intervalo):
    a = intervalo[0]
    b = intervalo[1]
    
    norma_error_absoluto = error_globlal_aprox_continuo(f, P, intervalo)**(1/2)
    
    integrand = lambda x: f(x)**2
    r,err = quad(integrand, a, b)
    norma_f = r**(1/2)
    
    return norma_error_absoluto/norma_f

In [27]:
def info_errores_aprox_continuo(f,P,intervalo):
    print("ERRORES\n")
    print("- error global:", error_globlal_aprox_continuo(f, P, intervalo))
    print("- error relativo %:", error_relativo_aprox_continuo(f, P, intervalo) *100,"%")

### 1.2.3 Ejemplos

#### Ejemplo 1

In [28]:
intervalo = [0,1]
grado = 2
f = lambda x: np.sin(np.pi * x)

phi,A,b,c,P = aproximacion_cm_continua(f, grado, intervalo)

info_aproximacion_cm(A, b, c, phi)
info_errores_aprox_continuo(f,P,intervalo)


Ajk:
|            1.0   	            0.5   	        0.33333   	|
|            0.5   	        0.33333   	           0.25   	|
|        0.33333   	           0.25   	            0.2   	|


bk:
|0.63662   0.31831   0.1893   |


C_k:
|-0.05047   4.12251   -4.12251   |


APROXIMACION POR CUADRADOS MÍNIMOS:

-4.1225116208762*x**2 + 4.1225116208762*x - 0.0504654977784531


ERRORES

- error global: 0.8008775387592536
- error relativo %: 126.56046292260893 %
